In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
%matplotlib inline

In [6]:
start = datetime.datetime(2006, 1, 1)
end = datetime.datetime(2016, 1, 1)

# Bank of America
BAC = web.DataReader("BAC", 'yahoo', start, end)

# CitiGroup
Citi = web.DataReader("C", 'yahoo', start, end)

# Goldman Sachs
GS = web.DataReader("GS", 'yahoo', start, end)

# JPMorgan Chase
JPM = web.DataReader("JPM", 'yahoo', start, end)

# Morgan Stanley
MS = web.DataReader("MS", 'yahoo', start, end)

# Wells Fargo
WellFC = web.DataReader("WFC", 'yahoo', start, end)

In [7]:
# Could also do this for a Panel Object
crashdf = web.DataReader(['BAC', 'C', 'GS', 'JPM', 'MS', 'WFC'],'yahoo', start, end)

In [8]:
# List of Keys as references to all Banks used in this analysis in alphabetical order
tickers = ['BAC', 'Citi', 'GS', 'JPM', 'MS', 'WFC']

In [ ]:
#Concatenating all data frames in one single called "bank_stocks
bank_stocks = pd.concat([BAC, Citi, GS, JPM, MS, WellFC],axis=1,keys=tickers)
bank_stocks.columns.names = ['Bank Ticker','Stock Info']

bank_stocks.head(5)

In [ ]:
#Setting a dataframe with only stock prices at close
closedf = pd.DataFrame()
for tick in tickers:
    closedf[tick+' Close Stock Value'] = bank_stocks[tick]['Close']
closedf.head(10)

In [ ]:
closedf.info()

In [ ]:
import seaborn as sns
sns.pairplot(closedf)

# EDA

Let's explore the data a bit! Before continuing, I encourage you to check out the documentation on [Multi-Level Indexing](http://pandas.pydata.org/pandas-docs/stable/advanced.html) and [Using .xs](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.xs.html).
Reference the solutions if you can not figure out how to use .xs(), since that will be a major part of this project.

In [ ]:
#The max Close price for each bank's stock throughout 2006-2016
bank_stocks.xs(key='Close',axis=1,level='Stock Info').max()


** Now, we´re going to create a new empty DataFrame called 'returnsdf'. This dataframe will contain the returns for each bank's stock.** 

**Returns are typically defined as:**

$$r_t = \frac{p_t - p_{t-1}}{p_{t-1}} = \frac{p_t}{p_{t-1}} - 1$$

&nbsp;
&nbsp;
<p style="text-align: center;">**Not counting Dividends and Stock Splits**</p>

In [ ]:
returnsdf = pd.DataFrame()

** And then, we can use pandas pct_change() metho on the Close column to create a column representing this return value in a loop to set this for all banks.**

In [ ]:
for tick in tickers:
    returnsdf[tick+' Return'] = bank_stocks[tick]['Close'].pct_change()
returnsdf.head(10)

**Note it that as expected, the 1st row received a null value. So to plot this, we need to keep in mind to skip the 1st row**

In [ ]:
#returns[1:]
sns.pairplot(returnsdf[1:])

In [ ]:
# Worst Drop (4 of them on Inauguration day)
returnsdf.idxmin()

** You should have noticed that Citigroup's largest drop and biggest gain were very close to one another, did anythign significant happen in that time frame? **

&nbsp;
[Citigroup had a stock split.](https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=citigroup+stock+2011+may)

In [ ]:
# Best Single Day Gain
# citigroup stock split in May 2011, but also JPM day after inauguration.
returnsdf.idxmax()

** Take a look at the standard deviation of the returns, which stock would you classify as the riskiest over the entire time period? Which would you classify as the riskiest for the year 2015?**

In [ ]:
returnsdf.std() # Citigroup riskiest

In [ ]:
returnsdf.loc['2015-01-01':'2015-12-31'].std()# Very similar risk profiles, but Morgan Stanley or BofA

** Create a distplot using seaborn of the 2015 returns for Morgan Stanley **

In [ ]:
sns.distplot(returnsdf.loc['2015-01-01':'2015-12-31']['MS Return'],color='green',bins=100)

** Create a distplot using seaborn of the 2008 returns for CitiGroup **

In [ ]:
sns.distplot(returnsdf.ix['2008-01-01':'2008-12-31']['Citi Return'],color='red',bins=100)

# More Visualization

A lot of this project will focus on visualizations. Feel free to use any of your preferred visualization libraries to try to recreate the described plots below, seaborn, matplotlib, plotly and cufflinks, or just pandas.

### Imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Optional Plotly Method Imports
import plotly
import cufflinks as cf
cf.go_offline()

** Creating a line plot showing close price for each bank for the entire index of time

In [ ]:
# plotly
bank_stocks.xs(key='Close',axis=1,level='Stock Info').plot(figsize=(15,5))

## Moving Averages

Let's analyze the moving averages for these stocks in the year 2008. 

** We will show the rolling 30 day average against the Close Price for Bank Of America's stock for the year 2008**

In [ ]:
plt.figure(figsize=(15,6))
BAC['Close'].ix['2008-01-01':'2009-01-01'].rolling(window=30).mean().plot(label='30 Day Avg')
BAC['Close'].ix['2008-01-01':'2009-01-01'].plot(label='BAC CLOSE')
plt.legend()
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

** Creating a heatmap of the correlation between the stocks Close Price.**

In [ ]:
sns.heatmap(bank_stocks.xs(key='Close',axis=1,level='Stock Info').corr(),annot=True)

** Optional: Using seaborn's clustermap to cluster the correlations together:**

In [ ]:
sns.clustermap(bank_stocks.xs(key='Close',axis=1,level='Stock Info').corr(),annot=True)

# Part 2

In this second part we will rely on the cufflinks library to create some Technical Analysis plots. This part of the project is experimental due to its heavy reliance on the cuffinks project, so it may be broken in the future.

** Using .iplot(kind='candle) to create a candle plot of Bank of America's stock from Jan 1st 2015 to Jan 1st 2016.**

In [ ]:
BAC[['Open', 'High', 'Low', 'Close']].ix['2015-01-01':'2016-01-01'].iplot(kind='candle')

** Using .ta_plot(study='sma') to create a Simple Moving Averages plot of Morgan Stanley for the year 2015.**

In [ ]:
MS['Close'].ix['2015-01-01':'2016-01-01'].ta_plot(study='sma',periods=[13,21,55],title='Simple Moving Averages')

**Using .ta_plot(study='boll') to create a Bollinger Band Plot for Bank of America for the year 2015.**

In [ ]:
BAC['Close'].ix['2015-01-01':'2016-01-01'].ta_plot(study='boll')